In [5]:
import os
import re

def parse_folder_name(folder_name):
    # Example folder name: CMSSW_14_1_NONLTO_X_2024-07-09-2300 or CMSSW_14_1_X_2024-07-09-2300
    match = re.match(r'^(CMSSW_\d+_\d+)(_?[^_]+)?(_X)?_(\d{4}-\d{2}-\d{2}-\d{4})$', folder_name)
    if not match:
        print(f"Folder name '{folder_name}' doesn't match the expected pattern of CMSSW_version_flavor_X_date-time")
        return None
    version = match.group(1)  # CMSSW_13_2 or CMSSW_14_1, etc.
    flavor_part = match.group(2) if match.group(2) else ""  # Any additional part like _GPU, _NONLTO, etc.
    flavor = flavor_part.lstrip('_')  # Strip the leading underscore if present
    if not flavor:  # If there's no flavor part, use 'DEFAULT'
        flavor = "DEFAULT"
    elif flavor == "X":
        flavor = "DEFAULT"  # Rename X to DEFAULT
    date = match.group(4)  # 2024-07-01-1100, etc.
    return version, flavor, date

def extract_and_parse_folders(directory):
    # Dictionary to store the parsed folder names grouped by version
    parsed_folders = {}
    
    # Loop through the items in the given directory
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            parsed_result = parse_folder_name(item)
            if parsed_result:
                version, flavor, date = parsed_result
                if version not in parsed_folders:
                    parsed_folders[version] = []
                parsed_folders[version].append((flavor, date))
    
    return parsed_folders

def create_classified_folders(parsed_folders, base_directory):
    classified_base = os.path.join(base_directory, 'IBs-Classified')
    
    # Create the base classified directory if it doesn't exist
    os.makedirs(classified_base, exist_ok=True)
    
    for version, flavor_date_list in parsed_folders.items():
        # Create the version directory
        version_dir = os.path.join(classified_base, version)
        os.makedirs(version_dir, exist_ok=True)
        
        # Group dates first
        dates = {}
        for flavor, date in flavor_date_list:
            if date not in dates:
                dates[date] = []
            dates[date].append(flavor)
        
        # Create date folders and flavor subfolders inside each date folder
        for date, flavors in dates.items():
            date_dir = os.path.join(version_dir, date)
            if not os.path.exists(date_dir):
                os.makedirs(date_dir, exist_ok=True)
            
            for flavor in flavors:
                if flavor == "X":
                    flavor = "DEFAULT"  # Rename X to DEFAULT
                flavor_dir = os.path.join(date_dir, flavor)
                if not os.path.exists(flavor_dir):
                    os.makedirs(flavor_dir, exist_ok=True)

def main():
    directory = '../Desktop/IBs'  # Replace with the path to your directory
    parsed_folders = extract_and_parse_folders(directory)
    
    # Create the classified folders based on parsed folder names
    create_classified_folders(parsed_folders, directory)
    
    print("Folders have been created and classified.")

if __name__ == "__main__":
    main()


Folder name 'IBs-Classified' doesn't match the expected pattern of CMSSW_version_flavor_X_date-time
Folders have been created and classified.
